In [21]:
import sys
import pulp

In [22]:
#Rooms and sizes
rooms = {
    'A': 100,
    'B': 50,
    'C': 50, 
}
#talks duration
talks = {
    'Python': 30, 
    'Java' : 30, 
    'C#' :60,
    'Java' :90, 
}
#Available slots: Room, start time and duration: total time:150 min
slots = [
    ('A', 0, 30),
    ('A',30, 60),
    ('A',90, 60),    
    ('B', 0, 30),
    ('B',30, 90),
    ('B',120, 30),
    ('C', 0, 90),
    ('C',90, 60),

]

In [23]:
#Proble definition
problem =pulp.LpProblem ('Conference Scedule', sense=pulp.LpMaximize)

In [24]:
#Variables
assign = {
    (talk, slot):pulp.LpVariable('%r in slot %r' %
                                 (talk, slot),
                                 cat = pulp.LpBinary)
    for talk in talks
                                 
    for slot in slots
}
                                 
#Print(assign)                                 

In [25]:
#Constraints
#add slot constraints
for slot in slots:
    #each slot may only be assigned at most onece
    #Only assign one talk per slot
    problem.addConstraint(
    sum(assign[(talk,slot)]
       for talk in talks)
    <=1)

In [26]:
#Assign all talks
#Add talk contraints
for talk in talks:
        #all talks have to be assigned
    problem.addConstraint(
        sum(assign[(talk,slot)]
            for slot in slots)
        ==1)
    problem.addConstraint(
        sum(slot[2]* assign[(talk,slot)]
            for slot in slots)
        == talks[talk])
    #Talks must fit the talk slots

In [7]:
#NONONONO Constraints
#Talk mus start later.Speaker not always available
talk = 'Python'
for slot in slots:
    if slot[1]<90:
        problem.addConstraint(
            assign[(talk,slot)] ==0)

In [8]:
#More than one talk per speaker
#Two talks given by the same person
talks_same_speaker = ('C#',
                      'Java',
                     )
#REPETIDO Available slots: Room, start time and duration: total time:150 min
#slots = [
#    ('A', 0, 30),
#    ('A',30, 60),
#    ('A',90, 60),    
#   ('B', 0, 30),
#    ('B',30, 90),
#    ('B',120, 30),
#    ('C', 0, 90),
#    ('C',90, 60),
#
#]

In [9]:
#Define some helper mappings
#Blocks: slots are arranged in a fixed 30 min block schedule; 
#each block is mapped to a slot in the schedule
blocks ={
    (room,start):None
    for room in sorted(rooms)
    for start in range(0,150,30)
}

#Block assigment variables
block_assign = {
    (talk, block): pulp.LpVariable('%r in block %r' %
                                  (talk, block),
                                  cat=pulp.LpBinary)
    for talk in talks
    for block in blocks
}

In [10]:
#Find blocks per slot
blocks_per_slot = {}
for slot in slots:
    room, start, duration = slot
    slot_blocks = []
    for block_time in range(start, start + duration, 30):
        blocks[(room, block_time)] = slot
        slot_blocks.append((room,block_time))
    blocks_per_slot[slot] = slot_blocks

In [11]:
#Find blocks per start time
time_blocks = {
    start:[(room, start)
          for room in sorted(rooms)]
    for start in range(0, 150, 30)
}

In [12]:
#Add slot contraints.  Link slots and blocks
for slot in slots:
    #Each slot may only be assigned at most once
    problem.addConstraint(
        sum(assign[(talk, slot)]
           for talk in talks)
    <=1)
    #Tie the block to the corresponding slots
    for block in blocks_per_slot[slot]:
        for talk in talks:
            problem.addConstraint(
            assign[(talk, slot)] == block_assign[(talk, block)])

In [13]:
#REPETIDO Two talks given by the same person
#talks_same_speaker = (
#    'C#',
#    'Java',
#    )
for blocks_per_time in time_blocks.values():
    problem.addConstraint(
        sum(block_assign[(talk,block)]
               for talk in talks_same_speaker
               for block in blocks_per_time)
        <=1)

In [28]:
#Define objective function & run solver
solver = None
problem.solve=(solver)
#Check Status
if problem.status == 1:
    print('found a solution. \n')
else: 
    print('failed %s' % pulp.LpStatus[problem.status])
    sys.exit(1)

failed Not Solved


SystemExit: 1